In [45]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [46]:
df = pd.read_csv("data/ham_spam.txt", sep="\t", names=['label', 'message'])

In [47]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [48]:
df['spam'] = pd.get_dummies(df['label'], drop_first=True)

In [49]:
df.drop(columns=['label'], inplace=True)
df.head()

,message,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [50]:
from sklearn.model_selection import train_test_split

In [51]:
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['spam'], test_size=0.3, random_state=45)

In [52]:
X_train

1201             Otherwise had part time job na-tuition..
77      I like you peoples very much:) but am very shy...
1346      Que pases un buen tiempo or something like that
2753                           Sat right? Okay thanks... 
3097    We walked from my moms. Right on stagwood pass...
                              ...                        
4473      3. You have received your mobile content. Enjoy
580     Arngd marriage is while u r walkin unfortuntly...
163     I'm so in love with you. I'm excited each day ...
4703                                           Anytime...
3616    I enjoy watching and playing football and bask...
Name: message, Length: 3900, dtype: object

In [73]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [54]:
help(TextVectorization)

Help on class TextVectorization in module tensorflow.python.keras.layers.preprocessing.text_vectorization:

class TextVectorization(tensorflow.python.keras.engine.base_preprocessing_layer.CombinerPreprocessingLayer)
 |  TextVectorization(*args, **kwargs)
 |  
 |  Text vectorization layer.
 |  
 |  This layer has basic options for managing text in a Keras model. It
 |  transforms a batch of strings (one sample = one string) into either a list of
 |  token indices (one sample = 1D tensor of integer token indices) or a dense
 |  representation (one sample = 1D tensor of float values representing data about
 |  the sample's tokens).
 |  
 |  If desired, the user can call this layer's adapt() method on a dataset.
 |  When this layer is adapted, it will analyze the dataset, determine the
 |  frequency of individual string values, and create a 'vocabulary' from them.
 |  This vocabulary can have unlimited size or be capped, depending on the
 |  configuration options for this layer; if there a

In [55]:
vectorization_layer = TextVectorization(
    max_tokens=None, standardize="lower_and_strip_punctuation",
    split="whitespace", ngrams=None, output_mode="int",
    output_sequence_length=None, pad_to_max_tokens=True
)

In [56]:
vectorization_layer.adapt(X_train.to_numpy())

In [57]:
print("1201 ---> ",vectorization_layer.get_vocabulary()[1201])
print(" 313 ---> ",vectorization_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorization_layer.get_vocabulary())))

1201 --->  custcare
 313 --->  into
Vocabulary size: 7922


In [58]:
from tensorflow.keras.layers import Dense, Activation,Dropout

embedding_dim = 16
max_features = 7922

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))

model.add(vectorization_layer)

model.add(layers.Embedding(max_features + 1, embedding_dim))
model.add(layers.Dropout(0.2))

model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dropout(0.2))

model.add(layers.Dense(1))

model.add(Dense(units=1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_2 (TextVe (None, None)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 16)          126768    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [59]:
epochs = 100
history = model.fit(
    x=X_train,y=y_train,
    validation_data=(X_test, y_test),
    verbose=1,
    epochs=epochs)

Epoch 1/100
122/122 [==============================] - 0s 2ms/step - loss: 0.6518 - binary_accuracy: 0.1341 - val_loss: 0.5800 - val_binary_accuracy: 0.1340
Epoch 2/100
122/122 [==============================] - 0s 1ms/step - loss: 0.4670 - binary_accuracy: 0.1341 - val_loss: 0.3690 - val_binary_accuracy: 0.1340
Epoch 3/100
122/122 [==============================] - 0s 1ms/step - loss: 0.3289 - binary_accuracy: 0.1341 - val_loss: 0.3094 - val_binary_accuracy: 0.1340
Epoch 4/100
122/122 [==============================] - 0s 1ms/step - loss: 0.2892 - binary_accuracy: 0.1341 - val_loss: 0.2898 - val_binary_accuracy: 0.1340
Epoch 5/100
122/122 [==============================] - 0s 1ms/step - loss: 0.2668 - binary_accuracy: 0.1341 - val_loss: 0.2699 - val_binary_accuracy: 0.1340
Epoch 6/100
122/122 [==============================] - 0s 1ms/step - loss: 0.2410 - binary_accuracy: 0.1341 - val_loss: 0.2486 - val_binary_accuracy: 0.1340
Epoch 7/100
122/122 [==============================] - 0s 

In [60]:

loss, accuracy = model.evaluate(X_test, y_test)
print(loss)
print(accuracy)

53/53 [==============================] - 0s 625us/step - loss: 0.1368 - binary_accuracy: 0.1340
0.13683274388313293
0.13397128880023956


In [61]:
y_pred = model.predict_classes(X_test)

In [62]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[1440    8]
 [  21  203]]


In [63]:
model.predict_classes(["How are you doing"])

array([[0]], dtype=int32)

## Saving model

In [70]:
model.save('tf_model') 

INFO:tensorflow:Assets written to: tf_model/assets


In [71]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("tf_model/") # path to the SavedModel directory
tflite_model = converter.convert()
# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

InvalidArgumentError: Cannot convert a Tensor of dtype resource to a NumPy array.